In [ ]:
# start from 7. image production in cookbook



# 1 create qpb images

In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
# onserver: mos_exspec_all_allfov.py, esas_exspec_all_allfov.sh


import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]
homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        for sregname in ['allfov']:
            regname = f'{srcname}_{sregname}'

            with open(f'mos_exspec_{sregname}.sh', 'w') as f:
                f.write(
f'''
heainit
sasinit21
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt


#### MOS ####
mkdir -p {regname}
mkdir -p {regname}/{regname}/
mkdir -p {regname}/img/
mkdir -p {regname}/logs/
mkdir -p {regname}/diagnose/
''')
                for E_idx in range(len(ELO_set)):

                    ELO = ELO_set[E_idx]
                    EHI = EHI_set[E_idx]
                        
                    f.write(
f"""
mosspectra eventfile={mos1file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=no elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_mos1.log
mosback inspecfile={mos1file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" 2>&1 |tee {regname}/logs/mosback_mos1.log 
mosspectra eventfile={mos2file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=no elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_mos2.log
mosback inspecfile={mos2file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}" 2>&1 |tee {regname}/logs/mosback_mos2.log 

for name in mos1 mos2
do
echo $name
mv $name*-{ELO}-{EHI}* {regname}/img
mv $name*.qdp {regname}/diagnose
mv $name*imt* {regname}/img
mv $name*imspdet* {regname}/img
done

mv mos1*-fovt.pi {regname}/{regname}/mos1S001-obj-{regname}.pi
mv mos1*-bkg.pi {regname}/{regname}/mos1S001-back-{regname}.pi
mv mos1*.arf {regname}/{regname}/mos1S001-{regname}.arf
mv mos1*.rmf {regname}/{regname}/mos1S001-{regname}.rmf

mv mos2*-fovt.pi {regname}/{regname}/mos2S002-obj-{regname}.pi
mv mos2*-bkg.pi {regname}/{regname}/mos2S002-back-{regname}.pi
mv mos2*.arf {regname}/{regname}/mos2S002-{regname}.arf
mv mos2*.rmf {regname}/{regname}/mos2S002-{regname}.rmf
""")



In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
# onserver: pn_exspec_all_allfov.py, esas_exspec_all_allfov.sh

import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        for sregname in ['allfov']:
            regname = f'{srcname}_{sregname}'

            with open(f'pn_exspec_{sregname}.sh', 'w') as f:
                f.write(
f'''

#### PN ####
heainit
sasinit20
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt
export sname=pn
export name={pnname}
cp $sname*-allevc.fits $name-clean.fits
cp $sname*-allevcoot.fits $name-clean-oot.fits
cp $sname*-corevc.fits $name-corn.fits
cp $sname*-corevcoot.fits $name-corn-oot.fits
cp $sname-bkgregtdet.fits $name-bkg_region-sky.fits 
cp $sname-bkgregtsky.fits $name-bkg_region-det.fits 
''')

                for E_idx in range(len(ELO_set)):

                    ELO = ELO_set[E_idx]
                    EHI = EHI_set[E_idx]
                    f.write(
f'''
for PAT in 0 4
do
pn-spectra prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb region=none mask=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 2>&1 |tee {regname}/logs/pnspectra_$PAT.log 
pn_back prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 2>&1 |tee {regname}/logs/pnback_$PAT.log

mkdir -p {regname}/img/pn_$PAT
mkdir -p {regname}/diagnose/pn_$PAT

mv pn*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv pn*-obj.pi {regname}/{regname}/pnS003-$PAT-obj-{regname}.pi
mv pn*-obj-oot.pi {regname}/{regname}/pnS003-$PAT-obj-oot-{regname}.pi
mv pn*-back.pi {regname}/{regname}/pnS003-$PAT-back-{regname}.pi
mv pn*.qdp {regname}/diagnose/pn_$PAT
mv pn*imt* {regname}/img/pn_$PAT
mv pn*.arf {regname}/{regname}/pnS003-$PAT-{regname}.arf
mv pn*.rmf {regname}/{regname}/pnS003-$PAT-{regname}.rmf
mv pn*imspdet* {regname}/img/pn_$PAT

done
''')



## 1.1 tidy the plots to use

In [ ]:
# onserver: tidy_plots.py

import os
from glob import glob
from tqdm import tqdm


homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')
for homepath in tqdm(homepaths):
    id = homepath.split('/')[-1]
    if os.path.isdir(homepath):
        imgpath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/{id}/{id}_allfov'
        savepath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/{id}/{id}_allfov/final_img'
        os.makedirs(savepath, exist_ok=True)

        # for mos
        os.system(f'cp {imgpath}/img/*bkgimdet*.fits {savepath}')
        os.system(f'cp {imgpath}/img/*fovimsky*.fits {savepath}')
        os.system(f'cp {imgpath}/img/*expimsky*.fits {savepath}')

        # for pn, take img that pat<=4
        
        os.system(f'cp {imgpath}/img/pn_4/*back*.fits {savepath}')
        os.system(f'cp {imgpath}/img/pn_4/*obj-im*.fits {savepath}')
        os.system(f'cp {imgpath}/img/pn_4/*exp-im*.fits {savepath}')
        os.system(f'cp {imgpath}/img/pn_4/*mask-im*.fits {savepath}')

        

## 1.2 rename pn images as sas21 format

In [ ]:
# onserver: rename_img.py

import os
from glob import glob
from tqdm import tqdm

def rename_files(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    oldnames_1 = ['back-im-det', 'obj-im-det', 'exp-im-det', 'mask-im-det']
    oldnames_2 = ['exp-im', 'obj-im', 'mask-im']
    newnames_1 = ['bkgimdet', 'fovimdet', 'expimdet', 'fovimtmask-det']
    newnames_2 = ['expimsky', 'fovimsky', 'fovimtmask']
    # Iterate through each file and rename based on specified patterns
    for filename in files:
        for i, oldname in enumerate(oldnames_1):
            if oldname in filename:
                new_filename = filename.replace(oldname, newnames_1[i])

                # Construct the full paths for the old and new filenames
                old_path = os.path.join(directory, filename)
                new_path = os.path.join(directory, new_filename)

                # Rename the file
                os.rename(old_path, new_path)
                print(old_path, new_path)
                
    files = os.listdir(directory)
    for filename in files:
        for i, oldname in enumerate(oldnames_2):
            if oldname in filename:
                new_filename = filename.replace(oldname, newnames_2[i])

                # Construct the full paths for the old and new filenames
                old_path = os.path.join(directory, filename)
                new_path = os.path.join(directory, new_filename)

                # Rename the file
                os.rename(old_path, new_path)
                print(old_path, new_path)

    print("Files renamed successfully.")

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')
for homepath in tqdm(homepaths):
    id = homepath.split('/')[-1]
    if os.path.isdir(homepath):
        savepath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/{id}/{id}_allfov/final_img'
        rename_files(savepath)


# 2 transform bkg img in det coords to sky coords
since there is a bug when making detpos.txt, update a fix
basically run the rotdet2sky twice, 1st record the command, 2nd run the recorded command and rerun rotdet2sky

In [1]:
# onserver: create_scripts_rot_det2sky_qpb_1st.py, rot_det2sky_qpb_all_1st.sh

def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')

ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]

for homepath in homepaths:
    id = homepath.split('/')[-1]
    if os.path.isdir(homepath):
        savepath = f'/data/yanling/XGAP-ABUN/data/alldata/XGAP/{id}/{id}_allfov/final_img'
        os.makedirs(f'{savepath}/logs', exist_ok=True)
        with open(f'{savepath}/../../rot_det2sky_qpb_1st.sh', 'w') as f:
            f.write(f'. set_sas.txt\n')
            f.write(f'. set_chips_on.txt\n')
            f.write(f'rm detpos.txt\n')
            for E_idx in range(len(ELO_set)):
                elo = ELO_set[E_idx]
                ehi = EHI_set[E_idx]

                name_dir = {'mos1':'mos1*-fovimsky*.fits', 'mos2':'mos2*-fovimsky*.fits', 'pn':'pn*-fovimsky*.fits'}

                name_lst = []
                prefix_lst = ''
                for file in glob(f'{savepath}/*0*-fovimsky*500-2000.fits'):
                    prefix = file.split('/')[-1].split('-')[0]
                    name_lst.append(prefix)

                name_lst = reorder_list(name_lst)

                for name in name_lst:
                    f.write(f'rotdet2sky intemplate={savepath}/{name}-fovimsky-{elo}-{ehi}.fits inimage={savepath}/{name}-bkgimdet-{elo}-{ehi}.fits outimage={savepath}/{name}-bkgimsky-{elo}-{ehi}.fits withdetxy=false withskyxy=false -V 10 2>&1 |tee {savepath}/logs/{name}-{elo}-{ehi}-rotdet2sky_1st.log\n')


In [ ]:
# onserver: create_scripts_rot_det2sky_qpb_2nd.py, rot_det2sky_qpb_all_2nd.sh
def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')

ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]

for homepath in homepaths:
    id = homepath.split('/')[-1]
    if os.path.isdir(homepath):
        savepath = f'{homepath}/{id}_allfov/final_img'
        with open(f'{savepath}/../../rot_det2sky_qpb_2nd.sh', 'w') as f:
            f.write(f'. set_sas.txt\n')
            f.write(f'. set_chips_on.txt\n')
            f.write(f'rm detpos.txt\n')
            for E_idx in range(len(ELO_set)):
                elo = ELO_set[E_idx]
                ehi = EHI_set[E_idx]

                name_dir = {'mos1':'mos1*-fovimsky*.fits', 'mos2':'mos2*-fovimsky*.fits', 'pn':'pn*-fovimsky*.fits'}

                name_lst = []
                prefix_lst = ''
                for file in glob(f'{savepath}/*0*-fovimsky*500-2000.fits'):
                    prefix = file.split('/')[-1].split('-')[0]
                    name_lst.append(prefix)

                name_lst = reorder_list(name_lst)

                for name in name_lst:
                    with open(f'{savepath}/logs/{name}-{elo}-{ehi}-rotdet2sky_1st.log') as f2:
                        lines = f2.readlines()
                        for line in lines:
                            if 'esky2det datastyle' in line:
                                f.write(f'esky2det {line.split("esky2det")[-1]}\n')
                    f.write(f'rotdet2sky intemplate={savepath}/{name}-fovimsky-{elo}-{ehi}.fits inimage={savepath}/{name}-bkgimdet-{elo}-{ehi}.fits outimage={savepath}/{name}-bkgimsky-{elo}-{ehi}.fits withdetxy=false withskyxy=false -V 10 2>&1 |tee {savepath}/logs/{name}-{elo}-{ehi}-rotdet2sky_2nd.log\n')
                    f.write(f'rm detpos.txt\n')

In [ ]:
# summary running
# onserver: rot_det2sky_qpb_all.sh in {homepath}
python rot_det2sky_qpb_1nd.py
. rot_det2sky_qpb_1st.sh
python rot_det2sky_qpb_2nd.py
. rot_det2sky_qpb_2nd.sh

# 3 proton (not applied yet)

## 3.1 fit a spec from lower SB regions in the field

# 4 comb image

In [ ]:
## onserver: comb_img_final_new.py  
## onserver: comb_img_all_new.sh

def reorder_list(input_list):
    order = ['mos1', 'mos2', 'pn']
    return sorted(input_list, key=lambda x: (order.index(x[:-4]), x[-4:]))

import os
from glob import glob
import numpy as np

homepaths = glob('/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG*')
ELO_set = [500, 2000, 500]
EHI_set = [2000, 8000, 8000]

bkgfracs = np.arange(0.5, 2.6, 0.1)

for homepath in homepaths:
    if os.path.isdir(homepath):
        id = homepath.split('/')[-1]
        savepath = f'{homepath}/{id}_allfov/final_img'
        os.chdir(savepath)
        os.system(f'rm -r {savepath}/science')
        os.system(f'rm -r {savepath}/show')
        os.makedirs(f'{savepath}/science', exist_ok=True)
        os.makedirs(f'{savepath}/show', exist_ok=True)
        with open(f'comb_img_new.sh', 'w') as f:
            for E_idx in range(len(ELO_set)):
                elo = ELO_set[E_idx]
                ehi = EHI_set[E_idx]

                name_dir = {'mos1':'mos1*-fovimsky*.fits', 'mos2':'mos2*-fovimsky*.fits', 'pn':'pn*-fovimsky*.fits'}

                name_lst = []
                prefix_lst = ''
                for file in glob(f'{savepath}/*0*-fovimsky*500-2000.fits'):
                    prefix = file.split('/')[-1].split('-')[0]
                    name_lst.append(prefix)

                name_lst = reorder_list(name_lst)
                
                for prefix in name_lst:
                    if 'mos' in prefix:
                        prefix_lst += f"{prefix[-5:]}  "
                    else:
                        prefix_lst += f"{prefix[-4:]}"

                f.write(f'''source ../../set_sas.txt \n''')
                f.write(f'''source ../../set_chips_on.txt \n''')

                for bkgfrac in bkgfracs:
                    for name in name_lst:
                        if 'mos' in name:
                            f.write(f'cp ../../{name[:4]}-bkgregtsky_{bkgfrac:.1f}_09fov.fits {name}-bkgregtsky.fits\n')
                            f.write(f'cp ../../{name[:4]}-bkgregtdet_{bkgfrac:.1f}_09fov.fits {name}-bkgregtdet.fits\n')
                            f.write(f'cp ../../{name}-cheeset_{bkgfrac:.1f}_09fov.fits {name}-cheeset.fits\n')
                        elif 'pn' in name:
                            f.write(f'cp ../../{name[:2]}-bkgregtsky_{bkgfrac:.1f}_09fov.fits {name}-bkgregtsky.fits\n')
                            f.write(f'cp ../../{name[:2]}-bkgregtdet_{bkgfrac:.1f}_09fov.fits {name}-bkgregtdet.fits\n')
                            f.write(f'cp ../../{name}-cheeset_{bkgfrac:.1f}_09fov.fits {name}-cheeset.fits\n')
                            f.write(f'cp {name}-fovimsky-{elo}-{ehi}-oot.fits {name}-fovimootsky-{elo}-{ehi}.fits\n')
                        else:
                            raise ValueError(f'Wrong inst name: {name}')

                    f.write(f'''combimage prefixlist="{prefix_lst}" withpartbkg=true withcheese=yes cheesetype=t elowlist={elo} ehighlist={ehi} 2>&1 |tee combimg.log  \n''')
                    f.write(f'''cp comb-fovimsky-{elo}-{ehi}.fits comb-fovimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits\n''')
                    f.write(f'''cp comb-expimsky-{elo}-{ehi}.fits comb-expimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits\n''')
                    f.write(f'''cp comb-bkgimsky-{elo}-{ehi}.fits comb-bkgimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits\n''')
                    f.write(f'farith comb-fovimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits comb-bkgimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits comb-netcts-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits / clobber=yes \n')
                    f.write(f'farith comb-netcts-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits comb-expimsky-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits comb-ctr-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits / clobber=yes \n')
                    f.write(f'fimgtrim comb-ctr-{elo}-{ehi}_{bkgfrac:.1f}_masked.fits threshlo=0 threshup=1e-3 const_lo=0 const_up=0 outfile=comb-ctr-{elo}-{ehi}_{bkgfrac:.1f}_masked_trimmed.fits clobber=yes\n')

                    # for science
                    f.write(f'cp comb-*.fits science/\n')

                    # for show 
                    f.write(f'binadapt prefix=comb elow={elo} ehigh={ehi} withpartbkg=true withmask=false withbinning=true binfactor=2 withsmoothing=true smoothcounts=50\n')
                    f.write(f'''
for file in *adaptimsky*.fits *sizeimsky*.fits *sigimsky*.fits *rateimsky*.fits *.qdp; do
cp $file show/${{file%.*}}_{bkgfrac:.1f}.${{file##*.}}
done\n''')


                

